<a href="https://colab.research.google.com/github/garbage38/Myocardial-Infraction-using-PPG-signal-analysis/blob/main/ppg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

# Data Augmentation function
def augment_data(X, y, noise_factor=0.02, shift_factor=0.05, scale_factor=0.1):
    augmented_X = []
    augmented_y = []

    for i in range(len(X)):
        signal = X[i]

        # Add random noise
        noise = np.random.normal(0, noise_factor, signal.shape)
        augmented_signal = signal + noise

        # Random time shift
        shift = int(np.random.uniform(-shift_factor * len(signal), shift_factor * len(signal)))
        augmented_signal = np.roll(augmented_signal, shift)

        # Random scaling
        scale = 1 + np.random.uniform(-scale_factor, scale_factor)
        augmented_signal = augmented_signal * scale

        augmented_X.append(augmented_signal)
        augmented_y.append(y[i])

    return np.array(augmented_X), np.array(augmented_y)

# Load dataset
data = pd.read_csv(r'/content/PPG_Dataset.csv')

# Step 1: Preprocess the data
X = data.iloc[:, :-1].values  # PPG signal values (0 to 1999 columns)
y = data.iloc[:, -1].values  # Labels (last column)

# Convert labels to binary (1 for MI, 0 for Normal)
y = np.where(y == 'MI', 1, 0)

# Reshape the data to have 3 dimensions: (samples, time_steps, features)
X = X.reshape(-1, 2000, 1)  # Shape (samples, 2000, 1)

# Normalize the data
scaler = StandardScaler()
X = np.array([scaler.fit_transform(x.reshape(-1, 1)) for x in X])  # Normalize each signal

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Data Augmentation for training set
X_train_augmented, y_train_augmented = augment_data(X_train, y_train)

# Step 4: Build an adjusted model
model = Sequential()

# 1st Convolutional Layer (increased filters)
model.add(Conv1D(64, kernel_size=5, activation='relu', input_shape=(2000, 1),
                 kernel_regularizer=regularizers.l2(0.001)))  # L2 regularization
model.add(Dropout(0.3))  # Increased dropout

# 2nd Convolutional Layer (added another convolutional layer)
model.add(Conv1D(128, kernel_size=5, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)))  # L2 regularization
model.add(Dropout(0.3))  # Dropout to prevent overfitting

# LSTM Layer (increased units)
model.add(LSTM(128, return_sequences=False, kernel_regularizer=regularizers.l2(0.001)))  # L2 regularization
model.add(Dropout(0.4))  # Dropout

# Fully Connected Dense Layer (increased units)
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))  # L2 regularization
model.add(Dropout(0.4))  # Dropout

# Output Layer: Sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Step 5: Compile the model
optimizer = Adam(learning_rate=0.0003)  # Increased learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])



# Step 7: Train the model
history = model.fit(
    X_train_augmented, y_train_augmented,
    epochs=20,  # Limit the number of epochs to prevent overfitting
    batch_size=64,
    validation_data=(X_test, y_test),

)

# Step 8: Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Step 9: Plot training history (accuracy and loss)
plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 238s 7s/step - accuracy: 0.5258 - loss: 1.0937 - val_accuracy: 0.8580 - val_loss: nan
Epoch 2/20
